This notebook presents a thorough description of the thermal model that has been developed in the framework of Gauthier Patin's PhD at the University of Amsterdam.

# Import

In [1]:
%pylab inline
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
from scipy import ndimage
import matplotlib.pyplot as plt

from ipywidgets import Layout, Button, Box, interact, interactive, fixed, interact_manual
import ipywidgets as wg
from IPython.display import display, clear_output

import Thermal_simulation_class
import Thermal_model
import Thermal_model_GIU

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Table of symbols 

| Symbol        | Unit        | Description  |
|:----------    |:------------|:-------------|
| $\alpha$      | m$^2$/s     | thermal diffusivity |
| $A_E$         | m$^2$       | area of east wall |
| $A_W$         | m$^2$       | area of west wall |
| $A_N$         | m$^2$       | area of north wall |
| $A_S$         | m$^2$       | area of south wall |
| $C_p$         | J/(kg K)    | specific heat capacity of paint |
| $dt$          | s           | timestep     |
| $dr$          | m           | width of cells   |
| $dz$          | m           | height of cells  |
| $h$           | W/(m$^2$K)  | Heat transfer coefficient from the top-facing surface   |
| $k$           | W/(m K)     | Thermal conductivity of a material   |
| $L$           | m           | height of the cylindrical system    |
| $M$           | kg          | mass of the paint material    |
| $N$           |             | number of degrees of freedom    |
| $n$_cols      |             | number of columns in the finite volume mesh |
| $n$_rows      |             | number of rows in the finite volume mesh |
| $R$           | m           | radius of the cylindrical system    |
| $\rho$        | kg/m$^3$    | mass density of paint    |
| $T_\infty$    | K           | far field temperature    |
| $T_{\rm air}$ | K           | temperature of the surrounding air  |
| $T_C$         | K           | temperature of the cell  |
| $T_S$         | K           | temperature at the surface of the paint  |
| $\vec{q}$     | W/m$^2$     | heat flux   |
| $Q$           | J           | heat        |
| $V$           | m$^3$       | volume of cells     |
| $\nabla T$    | K           | vector of temperature gradient  |
| $\Delta T$    | K           | temperature difference  |

# Description

## Figures 

<img src="../Figures/Finite_mesh.png" width="900px">

$$\Large{\text{Figure 1. Graphical representation of the finite mesh}}$$

***
<img src="../Figures/Neighbouring_cells.png" width="500px">

$$\Large{\text{Figure 2. Center cell and its neighbour}}$$

## Areas and volume 

$$\Large \text{Perimeter of a cicle} = \Large 2 \pi \space r $$

$$\Large \text{Area of a cicle} = \Large \pi \space r^2 $$

$$\Large \text{Volume of a cylinder} = \Large \pi \space r^2 \space h $$

$$ \text{where $r$ is the radius and $h$ the height.} $$

- - -

\begin{align}
\Large A_E &= \Large (2 \pi \space r_{j+1}) \space \Delta z \\
\Large A_W &= \Large (2 \pi \space r_{j}) \space \Delta z\\
\Large A_N &= \Large \pi \space (r_{j+1})^2 -\pi(r_j)^2\\
&= \Large 2\pi \space(r_{j+1}-r_j) \space \left[\frac{(r_{j+1} + r_j)}{2}\right]\\
&= \Large 2\pi\, \Delta r\, \left[\frac{(r_{j+1} + r_j)}{2}\right]\\
\Large A_S &= \Large 2\pi \space \Delta r \space \left[\frac{(r_{j+1} + r_j)}{2}\right]\\
\Large V &= \Large A_N \Delta z \\
&= \Large 2\pi\,\Delta z\,\Delta r\, \left[\frac{(r_{j+1} + r_j)}{2}\right]\\
&= \Large 2\pi\,\Delta z\,\Delta r\, r_{j+\frac{1}{2}}\\
\end{align}

# Initial conditions 

$T_\infty = T_{\rm air}$

$T(r,z,0) = T_\infty$



# Boundary conditions

**TOP**

$I = I_{in}(r) - fI_{in}(r)$

$I(r) = (1-f)I_{in}(r) = q^{light}$

$q_z = q^{conv} + q^{light} $

$q_z = h(\left. T\right|_{z=0} - T_\infty) + (1-f)I_{in}(r) $

**BOTTOM**

Dirichlet BC, $T = T_\infty$

**OUTER SIDE**

Dirichlet BC, $T = T_\infty$

**INNER SIDE**

$\left. q_r \right|_{r=0} = 0$

# Mathematical equations 

$$\large q = -k\nabla T$$ 

$$\large \nabla \equiv \frac{\partial}{\partial r}\hat{r} + \frac{\partial}{\partial \theta}\hat{\theta} + \frac{\partial}{\partial z}\hat{z}$$

$$\large \frac{\partial}{\partial \theta}\hat{\theta} = 0 $$

$$ \large q = \large -k\left[\frac{\partial T}{\partial r}\hat{r} + \frac{\partial T}{\partial z}\hat{z}\right]$$

$$\large q = h(T_S - T_\infty)$$

$$\large \Delta T = \frac{Q}{C_pM}$$

$$\large M = V\rho$$

$$\large \alpha = \frac{k}{\rho C_p}$$

The first equation in the cell just above corresponds to the Fourier's law of heat transfer. The law means that heat flows at temperature gradient, i.e. from hot to cold.

# Mathematical model

<center> Energy accumulated during timestep = Net total energy inflow during timestep

\begin{align}
\large Q &= \large q_{E}A_{E} + q_{W}A_{W} + q_{N}A_{N} + q_{S}A_{S} \\    
\large C_p M \Delta T &= \large q_{E}A_{E} + q_{W}A_{W} + q_{N}A_{N} + q_{S}A_{S} \\
\large \rho C_p V_{c}(T^{k+1}_{c} - T^{k}_{c}) &= \large q_{E}A_{E} + q_{W}A_{W} + q_{N}A_{N} + q_{S}A_{S} \\
&= \large k\left[\frac{(T^k_E - T^k_C)}{\Delta r}\right](\Delta z\,2\pi\,r_{j+1})\,\Delta t + k\left[\frac{T^k_W - T^k_C}{\Delta r}\right](\Delta z\,2\pi\,r_j)\,\Delta t + k\left[\frac{T^k_N - T^k_C}{\Delta z}\right] (\Delta r\,2\pi\,r_{j+\frac{1}{2}})\, \Delta t + k\left[\frac{T^k_S - T^k_C}{\Delta z}\right] (\Delta r\,2\pi\,r_{j+\frac{1}{2}})\, \Delta t \\
&= \large 2\pi\,\Delta t\,k \left[\left[\frac{(T^k_E - T^k_C)}{\Delta r}\right](\Delta z\,r_{j+1}) + \left[\frac{T^k_W - T^k_C}{\Delta r}\right](\Delta z\,r_j) + \left[\frac{T^k_N - T^k_C}{\Delta z}\right] (\Delta r\,r_{j+\frac{1}{2}}) + \left[\frac{T^k_S - T^k_C}{\Delta z}\right](\Delta r\,r_{j+\frac{1}{2}}) \right] \\
\large \rho C_p (2\pi\,\Delta z\,\Delta r\, r_{j+\frac{1}{2}})(T^{k+1}_{c} - T^{k}_{c}) &= \large 2\pi\,\Delta t\,k \left[\left[\frac{(T^k_E - T^k_C)}{\Delta r}\right](\Delta z\,r_{j+1}) + \left[\frac{T^k_W - T^k_C}{\Delta r}\right](\Delta z\,r_j) + \left[\frac{T^k_N - T^k_C}{\Delta z}\right] (\Delta r\,r_{j+\frac{1}{2}}) + \left[\frac{T^k_S - T^k_C}{\Delta z}\right](\Delta r\,r_{j+\frac{1}{2}}) \right] \\
\large (\Delta z\,\Delta r\, r_{j+\frac{1}{2}})(T^{k+1}_{c} - T^{k}_{c}) &= \large \frac{\Delta t\,k}{\rho C_p} \left[\left[\frac{(T^k_E - T^k_C)}{\Delta r}\right](\Delta z\,r_{j+1}) + \left[\frac{T^k_W - T^k_C}{\Delta r}\right](\Delta z\,r_j) + \left[\frac{T^k_N - T^k_C}{\Delta z}\right] (\Delta r\,r_{j+\frac{1}{2}}) + \left[\frac{T^k_S - T^k_C}{\Delta z}\right](\Delta r\,r_{j+\frac{1}{2}}) \right] \\
\large (T^{k+1}_{c} - T^{k}_{c}) &= \large \frac{\Delta t\,k}{\rho C_p} \left[\left[\frac{(T^k_E - T^k_C)}{\Delta r^2}\right]\Biggl(\frac{r_{j+1}}{r_{j+\frac{1}{2}}}\Biggl) + \left[\frac{T^k_W - T^k_C}{\Delta r^2}\right]\Biggl(\frac{r_j}{r_{j+\frac{1}{2}}}\Biggl) + \left[\frac{T^k_N - T^k_C}{\Delta z^2}\right] + \left[\frac{T^k_S - T^k_C}{\Delta z^2}\right]\right] \\
\large T^{k+1}_{c} &= \large T^{k}_{c} + \Delta t\,\alpha \left[\left[\frac{(T^k_E - T^k_C)}{\Delta r^2}\right]\Biggl(\frac{r_{j+1}}{r_{j+\frac{1}{2}}}\Biggl) + \left[\frac{T^k_W - T^k_C}{\Delta r^2}\right]\Biggl(\frac{r_j}{r_{j+\frac{1}{2}}}\Biggl) + \left[\frac{T^k_N - T^k_C}{\Delta z^2}\right] + \left[\frac{T^k_S - T^k_C}{\Delta z^2}\right]\right] 
\end{align}



We will write the time-stepping algorithm as $$\mathbf{T}^{k+1} = \mathbf{T}^{k} + \mathbf{M} \mathbf{T}^{k} + \mathbf{C},$$ where $\mathbf{T}$ is an $N$-dimensional vector of temperature degrees of freedom, superscript indicates timestep number, and $\mathbf{M}$ is an $N\times N$ matrix.

# Python code

The mathematical model described above has been written inside a python script called *Thermal_model.py*. This python file can be found in the Github repository in the *src* folder.

# GIU

In [2]:
model = Thermal_simulation_class.simulate()